In [1]:
#@title Install Packages

In [2]:
!pip install -q transformers
!pip install -q torchinfo
!pip install -q datasets
!pip install -q evaluate
!pip install -q nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-c

In [3]:
!sudo apt-get update
! sudo apt-get install tree

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [70.9 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,381 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu 

In [4]:
#@title Imports

import transformers
import evaluate

import nltk

from datasets import load_dataset
from torchinfo import summary

from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

import os
import pandas as pd
import numpy as np

In [5]:
# @title Mount Google Drive

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
dir_root = '/content/drive/MyDrive/266-final/'
# dir_data = '/content/drive/MyDrive/266-final/data/'
dir_data = '/content/drive/MyDrive/266-final/data/se21-t1-comp-lex-master/'
dir_models = '/content/drive/MyDrive/266-final/models/'
dir_results = '/content/drive/MyDrive/266-final/results/'

In [8]:
!tree -L 2 /content/drive/MyDrive/266-final/data/se21-t1-comp-lex-master/

/content/drive/MyDrive/266-final/data/se21-t1-comp-lex-master/
├── evaluate.py
├── Readme.md
├── test
│   ├── lcp_multi_test.tsv
│   └── lcp_single_test.tsv
├── test-labels
│   ├── lcp_multi_test.tsv
│   └── lcp_single_test.tsv
├── train
│   ├── lcp_multi_train.tsv
│   └── lcp_single_train.tsv
└── trial
    ├── lcp_multi_trial.tsv
    └── lcp_single_trial.tsv

4 directories, 10 files


In [9]:
# !tree -L 4 /content/drive/MyDrive/266-final/

In [10]:
!ls -R /content/drive/MyDrive/266-final/data/se21-t1-comp-lex-master/

/content/drive/MyDrive/266-final/data/se21-t1-comp-lex-master/:
evaluate.py  Readme.md	test  test-labels  train  trial

/content/drive/MyDrive/266-final/data/se21-t1-comp-lex-master/test:
lcp_multi_test.tsv  lcp_single_test.tsv

/content/drive/MyDrive/266-final/data/se21-t1-comp-lex-master/test-labels:
lcp_multi_test.tsv  lcp_single_test.tsv

/content/drive/MyDrive/266-final/data/se21-t1-comp-lex-master/train:
lcp_multi_train.tsv  lcp_single_train.tsv

/content/drive/MyDrive/266-final/data/se21-t1-comp-lex-master/trial:
lcp_multi_trial.tsv  lcp_single_trial.tsv


In [11]:
#@title Import Data

In [15]:
# # Construct file paths
# train_multi_path = os.path.join(dir_data, "train", "lcp_multi_train.tsv")
# train_single_path = os.path.join(dir_data, "train", "lcp_single_train.tsv")
# test_multi_path = os.path.join(dir_data, "test", "lcp_multi_test.tsv")
# test_single_path = os.path.join(dir_data, "test", "lcp_single_test.tsv")

# # Load them into pandas DataFrames
# df_train_multi = pd.read_csv(train_multi_path, sep="\t")
# df_train_single = pd.read_csv(train_single_path, sep="\t")
# df_test_multi = pd.read_csv(test_multi_path, sep="\t")
# df_test_single = pd.read_csv(test_single_path, sep="\t")

In [16]:
# print("Train Multi:\n", df_train_multi.head())
# print("Train Multi:\n", df_train_multi.shape)
# print("Train Multi:\n", df_train_multi.info())
# print("")
# print("\nTrain Single:\n", df_train_single.head())
# print("\nTrain Single:\n", df_train_single.shape)
# print("\nTrain Single:\n", df_train_single.info())
# print("")
# print("\nTest Multi:\n", df_test_multi.head())
# print("\nTest Multi:\n", df_test_multi.shape)
# print("\nTest Multi:\n", df_test_multi.info())
# print("")
# print("\nTest Single:\n", df_test_single.head())
# print("\nTest Single:\n", df_test_single.shape)
# print("\nTest Single:\n", df_test_single.info())

In [17]:
from datasets import load_dataset

# Suppose you just want to load train + test for your "single" dataset
data_files = {
    "train": os.path.join(dir_data, "train", "lcp_single_train.tsv"),
    "test":  os.path.join(dir_data, "test",  "lcp_single_test.tsv"),
}

dataset = load_dataset(
    "csv",  # we can still pass "csv" even though it's TSV
    data_files=data_files,
    delimiter="\t"  # crucial to handle TSV
)

# dataset is now a DatasetDict with 'train' and 'test' splits
print(dataset)

# Suppose each row has columns like `sentence`, `target_word`, `complexity`
# Next, define a tokenization function:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"  # for example
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def tokenize_function(examples):
    # Usually you'd tokenize the 'sentence' field (and maybe 'target_word'?).
    return tokenizer(examples["sentence"], truncation=True)

# Apply tokenization to each row in the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# If you want to train with a Trainer:
from transformers import DataCollatorWithPadding, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=1)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# etc. (TrainingArguments, Trainer, and so on).


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'corpus', 'sentence', 'token', 'complexity'],
        num_rows: 7232
    })
    test: Dataset({
        features: ['id', 'corpus', 'sentence', 'token', 'complexity'],
        num_rows: 808
    })
})


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Map:   0%|          | 0/7232 [00:00<?, ? examples/s]

Map:   0%|          | 0/808 [00:00<?, ? examples/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
# import os
# import pandas as pd
# from transformers import AutoTokenizer

# train_single_path = os.path.join(dir_data, "train", "lcp_single_train.tsv")
# df_train_single = pd.read_csv(train_single_path, sep="\t")

# # Inspect or preprocess in Pandas if you like
# print(df_train_single.head())

# # Suppose the file has columns: "id", "sentence", "target_word", "complexity"
# train_sentences = df_train_single["sentence"].tolist()
# train_targets = df_train_single["complexity"].tolist()  # e.g., a float label

# # Now tokenize
# model_checkpoint = "bert-base-cased"
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# encoded_train = tokenizer(
#     train_sentences,
#     truncation=True,
#     padding=True,      # or "max_length", "longest", or later w/DataCollator
#     return_tensors="pt"
# )
# # encoded_train now has input_ids, attention_mask, etc.


In [22]:
import os
import pandas as pd

# Assuming you've already defined:
# dir_data = '/content/drive/MyDrive/266-final/data/se21-t1-comp-lex-master/'

train_dir = os.path.join(dir_data, 'train')
test_dir = os.path.join(dir_data, 'test')


In [23]:
# # -- Single
# train_single_df = pd.read_csv(
#     os.path.join(train_dir, "lcp_single_train.tsv"),
#     sep="\t"
# )
# test_single_df = pd.read_csv(
#     os.path.join(test_dir, "lcp_single_test.tsv"),
#     sep="\t"
# )

# -- Multi
train_multi_df = pd.read_csv(
    os.path.join(train_dir, "lcp_multi_train.tsv"),
    sep="\t"
)
test_multi_df = pd.read_csv(
    os.path.join(test_dir, "lcp_multi_test.tsv"),
    sep="\t"
)


ParserError: Error tokenizing data. C error: EOF inside string starting at row 40